# Imports

In [118]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import STL10
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import numpy as np
import wandb

# Dataset

In [119]:
batch_size = 200

# Define the new image size
new_image_size = (64, 64)

# Create a new transformation that resizes the images
resize_transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert to a PIL image
    transforms.Resize(new_image_size),  # Resize the image to the new size
    transforms.ToTensor(),  # Convert back to a tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the image
])

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load STL-10 dataset
train_dataset = STL10(root='./data', split='train', transform=transform, download=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = STL10(root='./data', split='test', transform=transform, download=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Model

In [126]:
class Discriminator(nn.Module):
    def __init__(self, ngpu, dim_z, num_classes):
        self.dim_z = dim_z
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main= nn.Sequential(
            # input is ``(nc) x 64 x 64``
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf) x 32 x 32``
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*2) x 16 x 16``
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*4) x 8 x 8``
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*8) x 4 x 4``
            nn.Conv2d(ndf * 8, dim_z, 4, 1, 0, bias=False),
        )
        self.fc = nn.Linear(dim_z, num_classes)


    def forward(self, input):
        z = self.main(input)
        print(f'z.shape {z.shape}')
        
        #z = z.view(-1)
        
        c = self.fc(z)
        print(f'c.shape {c.shape}')
        
        return 

In [127]:
# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 64

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1


# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Instantiate the model
netD = Discriminator(ngpu=0, dim_z=64, num_classes=10)

# Criterion / Loss Function

In [128]:
criterion = nn.CrossEntropyLoss()

# Optimizer

In [129]:
optimizer = optim.Adam(netD.parameters(), lr=1e-4, betas= (0.9, 0.999), weight_decay=0.1)

# Training Loop

In [124]:
# set up wandb
wandb.login()
wandb.init(project="dcgan-project")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [130]:
# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Training loop
best_acc = float('-inf')
best_model_state = None

all_loss = []
all_accuracy = []
correct = 0
total = 0
epochs = 200

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(train_loader, 0):
        
        
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

        # Forward pass real batch through D
        print(f'real_cpu.shape {real_cpu.shape}')
        output = netD(real_cpu)
        
        # Calculate loss on all-real batch
        loss = criterion(output, label)
        all_loss.append(loss.item())


        # Calculate accuracy on all-real batch
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()
        accuracy = correct / total
        all_accuracy.append(accuracy)

        
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()        
        optimizer.step()

        # Log metrics to wandb
        wandb.log({"train_loss": loss.item(), "train_accuracy": accuracy})

    
    print(f"Epoch {epoch + 1}, Accuracy: {accuracy}, Loss: {loss.item()}")
    
    # Update best model if this epoch had the higest accuracy so far
    if accuracy > best_acc:
        best_acc = accuracy
        #print(f'best accuracy {best_acc}')
        best_model_state = model.state_dict()
    

# Load the best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print("Loaded the model with the highest accuracy.")

Starting Training Loop...
real_cpu.shape torch.Size([200, 3, 96, 96])
z.shape torch.Size([200, 64, 3, 3])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (38400x3 and 64x10)